### NOTES

##### - Desafio Titanic Kaggle (rev2.0)
##### - Resolução do canal DevVerso [BR]

---

### INITIAL IMPORTS 

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics, model_selection

sns.set_style('whitegrid')


-----

### FUNCTIONS

In [2]:
def sex_in_binary(value):
    if value == 'female':
        return 1
    else:
        return 0


In [3]:
def titanic_func(data, column, count=True):
    #print(f'{data[column]}')
    print(f'Quant. valores únicos: {data[column].nunique()}')
    print(f'Quais os valores únicos: {data[column].unique()}')
    print(f'Quant. valores nulos: {data[column].isnull().sum()}')
    print(f'Quant. por opção:\n{data[column].value_counts()}')
    if count == True:
        sns.countplot(data=data, x=column, hue='Survived')
    else:
        sns.displot(data[column], kde=True)


In [4]:
def accuracy_func(algo, x_train, y_train, cross_val):
    model = algo.fit(x_train, y_train)
    accuracy = round(model.score(x_train, y_train) * 100, 2)
    train_predict = model_selection.cross_val_predict(algo, x_train, y_train, cv = cross_val, n_jobs=-1)
    cross_val_accuracy = round(metrics.accuracy_score(y_train, train_predict) * 100, 2)
    return accuracy, cross_val_accuracy


---

#### CLEANING / EDA

In [5]:
#.info()  #.head(3)  #.tail()  #.columns
#[''].unique()  #.isnull().sum()  #[''].value_counts()
# #.describe()

#dataset treino
df_train = pd.read_csv('./train.csv')
#891 entradas
#null: Age 177; Cabin 687; Embarked 2

#dataset teste
df_test = pd.read_csv('./test.csv')
#418 entradas
#null: Age 86; Cabin: 327


In [6]:
#df_train fica somente p/ consulta
#alterações e criação de novas series são add no df e df_titanic
 
df = pd.DataFrame()

df_titanic = pd.concat([df_train, df_test], ignore_index=True)
#titanic_func(df_titanic, 'Fare', 0)
#titanic_func(df_titanic, 'Title')


In [7]:
df['survived'] = df_titanic['Survived']
df['p_class'] = df_titanic['Pclass']
df['sex'] = df_titanic['Sex'].map(sex_in_binary)


In [8]:
#NaN da coluna Age serão preenchidos com a média da classe
for i in df_titanic.index:
    if pd.isnull(df_titanic['Age'][i]):
        if df_titanic['Pclass'][i] == 1:
            df_titanic['Age'][i] = round(df_titanic[df_titanic['Pclass'] == 1]['Age'].mean(), 0)
        elif df_titanic['Pclass'][i] == 2:
            df_titanic['Age'][i] = round(df_titanic[df_titanic['Pclass'] == 2]['Age'].mean(), 0)
        else:
            df_titanic['Age'][i] = round(df_titanic[df_titanic['Pclass'] == 3]['Age'].mean(), 0)
    else:
        continue
    
df['age'] = df_titanic['Age']


C:\Users\edgar\AppData\Local\Temp\ipykernel_15916\1196738266.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_titanic['Age'][i] = round(df_titanic[df_titanic['Pclass'] == 3]['Age'].mean(), 0)
C:\Users\edgar\AppData\Local\Temp\ipykernel_15916\1196738266.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_titanic['Age'][i] = round(df_titanic[df_titanic['Pclass'] == 2]['Age'].mean(), 0)
C:\Users\edgar\AppData\Local\Temp\ipykernel_15916\1196738266.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-

In [9]:
df['sib_sp'] = df_titanic['SibSp']
df['par_ch'] = df_titanic['Parch']

#tamanho da família: 1 + sib_sp + par_ch
df['family_size'] = 1 + df['sib_sp'] + df['par_ch']


In [10]:
#df_titanic[df_titanic['Fare'].isnull()]
#df_titanic[df_titanic['Pclass'] == 3]['Fare'].mean() #13
df_titanic['Fare'].fillna(df_titanic[df_titanic['Pclass'] == 3]['Fare'].mean(), inplace=True)
df['fare'] = df_titanic['Fare']

# Pclass = 3 -> valor mais frequente


In [11]:
df_titanic['Embarked'].fillna('C', inplace=True)
df['embarked'] = df_titanic['Embarked']


In [12]:
#[df_titanic['Name'][n].split(',')[1].split('.')[0].strip() for n in df_titanic.index]
df_titanic['Title'] = df_titanic['Name'].apply(lambda name: name.split(',')[1].split('.')[0].strip())
df_titanic['Title'] = [n if n in ['Mr', 'Mrs', 'Miss', 'Master'] else 'Person' for n in df_titanic['Title']]
df['title'] = df_titanic['Title']


In [13]:
df

,survived,p_class,sex,age,sib_sp,par_ch,family_size,fare,embarked,title
0,0.0,3,0,22.0,1,0,2,7.2500,S,Mr
1,1.0,1,1,38.0,1,0,2,71.2833,C,Mrs
2,1.0,3,1,26.0,0,0,1,7.9250,S,Miss
3,1.0,1,1,35.0,1,0,2,53.1000,S,Mrs
4,0.0,3,0,35.0,0,0,1,8.0500,S,Mr
...,...,...,...,...,...,...,...,...,...,...
1304,NaN,3,0,25.0,0,0,1,8.0500,S,Mr
1305,NaN,1,1,39.0,0,0,1,108.9000,C,Person
1306,NaN,3,0,38.5,0,0,1,7.2500,S,Mr
1307,NaN,3,0,25.0,0,0,1,8.0500,S,Mr


In [13]:
#get_dummies() -> 
pclass = pd.get_dummies(df['p_class'], prefix='p_class', drop_first=True)
title = pd.get_dummies(df['title'], prefix='title', drop_first=True)
embarked = pd.get_dummies(df['embarked'], prefix='embarked', drop_first=True)

#drop_first=True -> ("se não for caso A, nem caso B, nem caso C, então só pode ser caso D")


In [14]:
titanic_full = pd.concat([df, pclass, title, embarked], axis=1)

titanic_full.drop(['p_class', 'title', 'embarked'], axis=1, inplace=True)

In [15]:
#separando o df_titanic em treino e teste
train_index = len(df_train) #891
test_index = len(df_titanic) - len(df_test) #891
titanic_train = titanic_full[:train_index].copy() #[:891]
titanic_test = titanic_full[test_index:].copy() #[891:]
titanic_train['survived'] = titanic_train['survived'].astype(int)


In [16]:
#variáveis de treino
x = titanic_train.drop('survived', axis=1)
y = titanic_train['survived']

#variável de teste
x_test = titanic_test.drop('survived', axis=1)

#x_train, x_test, y_train, y_test

---

#### TRAIN / TUNING / VALIDATION / TEST

In [17]:
#random forest
from sklearn.ensemble import RandomForestClassifier

acc_rf, acc_rf_cross_val = accuracy_func(RandomForestClassifier(), x, y, 10)
print('Random Forest')
print(f'Acurácia: {acc_rf}')
print(f'Acurácia Validação Cruzada: {acc_rf_cross_val}')


Random Forest
Acurácia: 98.32
Acurácia Validação Cruzada: 80.7


In [18]:
#logistic regression
from sklearn.linear_model import LogisticRegression

acc_lr, acc_lr_cross_val = accuracy_func(LogisticRegression(max_iter=1000), x, y, 10)
print('Logistic Regression')
print(f'Acurácia: {acc_lr}')
print(f'Acurácia Validação Cruzada: {acc_lr_cross_val}')


Logistic Regression
Acurácia: 82.94
Acurácia Validação Cruzada: 82.38


In [19]:
#k-nearest neighbours
from sklearn.neighbors import KNeighborsClassifier

acc_knn, acc_knn_cross_val = accuracy_func(KNeighborsClassifier(), x, y, 10)
print('KNN')
print(f'Acurácia: {acc_knn}')
print(f'Acurácia Validação Cruzada: {acc_knn_cross_val}')


KNN
Acurácia: 80.81
Acurácia Validação Cruzada: 72.39


In [20]:
#gaussian naive bayes
from sklearn.naive_bayes import GaussianNB

acc_gaussian, acc_gaussian_cross_val = accuracy_func(GaussianNB(), x, y, 10)
print('Gaussian Naive Bayes')
print(f'Acurácia: {acc_gaussian}')
print(f'Acurácia Validação Cruzada: {acc_gaussian_cross_val}')


Gaussian Naive Bayes
Acurácia: 78.79
Acurácia Validação Cruzada: 78.56


In [21]:
#linear support vector machines (SVC)
from sklearn.svm import LinearSVC

acc_lsvc, acc_lsvc_cross_val = accuracy_func(LinearSVC(dual=False), x, y, 10)
print('Support Vector Machines')
print(f'Acurácia: {acc_lsvc}')
print(f'Acurácia Validação Cruzada: {acc_lsvc_cross_val}')


Support Vector Machines
Acurácia: 83.28
Acurácia Validação Cruzada: 82.72


In [22]:
#stochastic gradient descent
from sklearn.linear_model import SGDClassifier

acc_sgd, acc_sgd_cross_val = accuracy_func(SGDClassifier(), x, y, 10)
print('Support Vector Machines')
print(f'Acurácia: {acc_sgd}')
print(f'Acurácia Validação Cruzada: {acc_sgd_cross_val}')


Support Vector Machines
Acurácia: 75.2
Acurácia Validação Cruzada: 72.95


In [23]:
#decision tree classifiers
from sklearn.tree import DecisionTreeClassifier

acc_dt, acc_dt_cross_val = accuracy_func(DecisionTreeClassifier(), x, y, 10)
print('Decision Tree Classifiers')
print(f'Acurácia: {acc_dt}')
print(f'Acurácia Validação Cruzada: {acc_dt_cross_val}')


Decision Tree Classifiers
Acurácia: 98.32
Acurácia Validação Cruzada: 78.79


In [24]:
#gradient boost classifier
from sklearn.ensemble import GradientBoostingClassifier

acc_gbt, acc_gbt_cross_val = accuracy_func(GradientBoostingClassifier(), x, y, 10)
print('Gradient Boost Classifier')
print(f'Acurácia: {acc_gbt}')
print(f'Acurácia Validação Cruzada: {acc_gbt_cross_val}')


Gradient Boost Classifier
Acurácia: 89.67
Acurácia Validação Cruzada: 83.05


In [25]:
#tuning
params = dict(
    max_depth = [n for n in range(1, 5)],
    min_samples_split = [n for n in range(2, 6)],
    min_samples_leaf = [n for n in range(2, 6)],
    n_estimators = [n for n in range(10, 50, 10)]
)


In [26]:
#cross validation
from sklearn.model_selection import GridSearchCV

gbc = GradientBoostingClassifier()
gbc_cross_val = GridSearchCV(estimator=gbc, param_grid=params, cv=10)
gbc_cross_val.fit(x, y) #
print('Grid Search Cross Validation')
print(f'Melhor pontuação: {gbc_cross_val.best_score_:.6f}')
print(f'Melhores parâmetros: {gbc_cross_val.best_estimator_}')


Grid Search Cross Validation
Melhor pontuação: 0.848514
Melhores parâmetros: GradientBoostingClassifier(max_depth=4, min_samples_leaf=2, n_estimators=30)


In [27]:
#teste
gradient_boosting_classifier_predict = gbc_cross_val.predict(x_test)


---

#### MODEL SUBMISSION

In [28]:
passenger_id = df_test['PassengerId']

sub = pd.Series(gradient_boosting_classifier_predict, index=passenger_id, name='Survived')
#sub.shape
sub.to_csv('./titanic_grad_boost_class.csv', header=True)


---

#### END